## 관광상품 리뷰 크롤링 분석

In [1]:
# only selenium
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.support.ui import WebDriverWait

from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By

import time

import pandas as pd

/var/folders/kg/xp1kxddd3r365qjsy8z_p9lh0000gn/T/ipykernel_2995/1640226374.py:12: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [11]:
# Selenium 웹 드라이버 서비스 설정, 실행
service = Service(executable_path=ChromeDriverManager().install())
driver = webdriver.Chrome(service=service)

url ='https://www.hanatour.com/trp/pkg/CHPC0PKG0200M200?pkgCd=AVP231240401VJA&prePage=major-products&directSale=PL00113904'
driver.get(url)

# 여행후기를 클릭 click
# review_link = driver.find_element(By.XPATH, '//*[@id="ulTag"]/li[6]/a')
# 좀 더 안전하게 기다린 후 크롤링
review_link = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.XPATH, '//*[@id="ulTag"]/li[6]/a')))
review_link.click()
time.sleep(2)

# 각 리뷰 정보를 저장할 빈 리스트
reviews = []

#li 태그들을 가져오는 부분
lis = driver.find_elements(By.CSS_SELECTOR, 'ul.list_review_v2 > li')
for li in lis:
    # 하나의 리뷰 정보를 저장할 딕셔너리
    review_info = {}

    # rating_info에서 각 정보를 추출
    rating_info = li.find_element(By.CLASS_NAME, 'rating_info')

    # 별점 : strong 태그 있으면 추출, 없으면 ""ArithmeticError
    rating = rating_info.find_element(By.TAG_NAME, 'strong').text if rating_info.find_element(By.TAG_NAME, 'strong') else ""
    review_info['rating'] = rating

    # span 태그들의 text를 차례차례 review_info의 각 항목에 저장
    spans = rating_info.find_elements(By.TAG_NAME, 'span')
    review_info['user'] = spans[1].text
    review_info['category'] = spans[2].text
    review_info['age'] = spans[3].text
    review_info['date'] = spans[-1].text

    # review_cont con 클래스 에서 리뷰 추출
    review = li.find_element(By.CLASS_NAME, 'review_cont')
    review_text = review.text if review else ""
    review_info['review'] = review.text

    # review_cate 클래스에서 리뷰 카테고리를 추출
    review_cate = li.find_element(By.CLASS_NAME, 'review_cate')
    review_cate_text = review_cate.text if review_cate else ""
    review_info['review_cate'] = review_cate_text

    reviews.append(review_info)

# 종료
driver.quit()

# 데이터프레임 생성
df = pd.DataFrame(reviews)
df

,rating,user,category,age,date,review,review_cate
0,5,yjo****,가족여행으로 좋아요,50대,2024.02.21,알찬구성으로 재밌고볼거리다양해요 너무타이트하지도않고 지루하지않게 잠깐잠깐의휴식이나자...,호텔이 관광지와 가까워요\n객실 뷰가 좋아요\n객실이 깨끗해요\n일정이 알차요\n식...
1,4,dhl***,가족여행으로 좋아요,40대,2024.02.19,어르신 모시고 여행하기 좋습니다. 워낙 관광객이 많은 곳이라 아침 일찍부터 저녁 늦...,일정이 알차요\n현지 음식이 맛있어요\n호텔이 관광지와 가까워요\n가격만큼 가치 있어요
2,4,아자,친구모임으로 좋아요,50대,2024.02.15,딸과 둘이 떠난 여행이라 더 뜻 깊었던 여행이었고 같이 여행한 팀원들도 모두 괜찮은...,호텔이 관광지와 가까워요\n일정이 여유로워요\n현지 음식이 맛있어요\n즐길 거리가 ...
3,5,qow*******,커플여행으로 좋아요,20대,2024.02.12,와이프와 결혼해서 처음가는 해외여행 기대반 설레임 반으로 하나투어 강혜원가이드님을 ...,일정이 알차요\n가이드가 전문적이에요\n가이드가 열정적이에요\n가이드가 배려 깊고 ...
4,4,ban******,가족여행으로 좋아요,40대,2024.02.12,즐거운 여행이었어요~,호텔이 관광지와 가까워요\n일정이 여유로워요\n현지 음식이 다양해요\n가이드가 전문...
5,5,piz*******,친구모임으로 좋아요,40대,2024.02.02,"가이드님 친절, 현지 대동하는 가이드님도 대박 친절. 적절한 옵션 진행은 좋았고, ...",객실이 깨끗해요\n호텔이 시내에 있어요\n현지 음식이 맛있어요\n가이드가 전문적이에요
6,5,css****,친구모임으로 좋아요,50대,2024.01.30,굿입니다 정말 좋아요 만족합니다 좋아요 부탁드립니다 여러분들의 의견을 수렴하고 불만...,객실이 깨끗해요\n일정이 알차요\n현지 음식이 맛있어요\n가이드가 재밌어요\n가격만...
7,5,min*******,가족여행으로 좋아요,10대,2024.01.29,✈️2022.08.10(수) ~ 2022.08.14(일)✈️ With. 박홍교 가이...,호텔에서 쉬기 좋아요\n일정이 알차요\n현지 음식이 다양해요\n가격이 합리적이에요\...
8,5,pp2*****,친구모임으로 좋아요,40대,2024.01.26,아빠 칠순기념으로 가족여행 다녀왔어요. 가기전에는 좀 걱정했는데 가이드님이 워낙 잘...,객실이 깨끗해요\n객실 뷰가 좋아요\n일정이 알차요\n현지 음식이 맛있어요\n가이드...
9,4,yoo*******,가족여행으로 좋아요,50대,2024.01.10,날씨가 너무 좋아서 여행하기가 굿이었네요. 가이드분두 전일정 여행 동반자들과의 원활...,객실이 깨끗해요\n호텔이 관광지와 가까워요\n호텔이 시내에 있어요\n객실 뷰가 좋아...


In [21]:
# Selenium 웹 드라이버 서비스 설정, 실행
service = Service(executable_path=ChromeDriverManager().install())
driver = webdriver.Chrome(service=service)

url ='https://www.hanatour.com/trp/pkg/CHPC0PKG0200M200?pkgCd=AVP231240401VJA&prePage=major-products&directSale=PL00113904'
driver.get(url)

# 여행후기를 클릭 click
# review_link = driver.find_element(By.XPATH, '//*[@id="ulTag"]/li[6]/a')
# 좀 더 안전하게 기다린 후 크롤링
review_link = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.XPATH, '//*[@id="ulTag"]/li[6]/a')))
review_link.click()
time.sleep(2)

# 각 리뷰 정보를 저장할 빈 리스트
reviews = []

for page_num in range(1, 13) : #1부터 12까지 반복합니다.
    # 페이지네이션할 요소를 찾습니다.
    # paginate_div = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CLASS_NAME, "pagenate")))
    # paginate_div = driver.find_element(By.CLASS_NAME, 'pagenate')
    paginate_div = driver.find_element(By.XPATH, '//*[@id="sticky06"]/div/div[4]/div[3]/div')
    
    # 현재 몇 페이지 크롤링 중인지, 출력
    print("현재 페이지: ",page_num)

    #li 태그들을 가져오는 부분
    lis = driver.find_elements(By.CSS_SELECTOR, 'ul.list_review_v2 > li')
    for li in lis:
        # 하나의 리뷰 정보를 저장할 딕셔너리
        review_info = {}

        # rating_info에서 각 정보를 추출
        rating_info = li.find_element(By.CLASS_NAME, 'rating_info')

        # 별점 : strong 태그 있으면 추출, 없으면 ""ArithmeticError
        rating = rating_info.find_element(By.TAG_NAME, 'strong').text if rating_info.find_element(By.TAG_NAME, 'strong') else ""
        review_info['rating'] = rating

        # span 태그들의 text를 차례차례 review_info의 각 항목에 저장
        spans = rating_info.find_elements(By.TAG_NAME, 'span')
        review_info['user'] = spans[1].text
        review_info['category'] = spans[2].text
        review_info['age'] = spans[3].text
        review_info['date'] = spans[-1].text

        # review_cont con 클래스 에서 리뷰 추출
        review = li.find_element(By.CLASS_NAME, 'review_cont')
        review_text = review.text if review else ""
        review_info['review'] = review.text

        # review_cate 클래스에서 리뷰 카테고리를 추출
        review_cate = li.find_element(By.CLASS_NAME, 'review_cate')
        review_cate_text = review_cate.text if review_cate else ""
        review_info['review_cate'] = review_cate_text

        reviews.append(review_info)
    # 다음 페이지로 이동 
    try :
        # 페이지가 1~9, 11~19, 21~29
        if page_num % 10 != 0:
            next_link = paginate_div.find_element(By.XPATH, f"//span/a[text()='{page_num+1}']")
            next_link.click()
        # 페이지가 10, 20, 30,...
        else:
            next_link = paginate_div.find_element(By.XPATH, '//*[@id="sticky06"]/div/div[4]/div[3]/div/div/a[3]')
            next_link.click()
        time.sleep(2)
    except :
        print("마지막 페이지에 도달했습니다.")
        break

# 종료
driver.quit()

# 데이터프레임 생성
df = pd.DataFrame(reviews)
df

현재 페이지:  1
현재 페이지:  2
현재 페이지:  3
현재 페이지:  4
현재 페이지:  5
현재 페이지:  6
현재 페이지:  7
현재 페이지:  8
현재 페이지:  9
현재 페이지:  10
현재 페이지:  11
현재 페이지:  12


,rating,user,category,age,date,review,review_cate
0,5,yjo****,가족여행으로 좋아요,50대,2024.02.21,알찬구성으로 재밌고볼거리다양해요 너무타이트하지도않고 지루하지않게 잠깐잠깐의휴식이나자...,호텔이 관광지와 가까워요\n객실 뷰가 좋아요\n객실이 깨끗해요\n일정이 알차요\n식...
1,4,dhl***,가족여행으로 좋아요,40대,2024.02.19,어르신 모시고 여행하기 좋습니다. 워낙 관광객이 많은 곳이라 아침 일찍부터 저녁 늦...,일정이 알차요\n현지 음식이 맛있어요\n호텔이 관광지와 가까워요\n가격만큼 가치 있어요
2,4,아자,친구모임으로 좋아요,50대,2024.02.15,딸과 둘이 떠난 여행이라 더 뜻 깊었던 여행이었고 같이 여행한 팀원들도 모두 괜찮은...,호텔이 관광지와 가까워요\n일정이 여유로워요\n현지 음식이 맛있어요\n즐길 거리가 ...
3,5,qow*******,커플여행으로 좋아요,20대,2024.02.12,와이프와 결혼해서 처음가는 해외여행 기대반 설레임 반으로 하나투어 강혜원가이드님을 ...,일정이 알차요\n가이드가 전문적이에요\n가이드가 열정적이에요\n가이드가 배려 깊고 ...
4,4,ban******,가족여행으로 좋아요,40대,2024.02.12,즐거운 여행이었어요~,호텔이 관광지와 가까워요\n일정이 여유로워요\n현지 음식이 다양해요\n가이드가 전문...
...,...,...,...,...,...,...,...
115,3,파이어스타,커플여행으로 좋아요,40대,2023.12.31,숙소는 만족합니다. 조식은 종류가 다양하나 맛있다는 느낌은 들지않았으나 부대시설로 ...,객실이 깨끗해요\n호텔 부대시설이 좋아요\n호텔에서 쉬기 좋아요\n객실 뷰가 좋아요...
116,1,son*******,친구모임으로 좋아요,20대,2023.12.31,하나투어 다낭여행 기대하고 갔는데 엄청 별로였어요. 그래도 하나 꼽아보자면 첫날에는...,호텔이 시내에 있어요
117,5,qaz****,친구모임으로 좋아요,30대,2023.12.27,이번 다낭/호이안 패키지 여행은 그동안 스트레스와 번아웃으로 지쳐있던 중에 충동적으...,객실이 깨끗해요\n객실 뷰가 좋아요\n일정이 알차요\n현지 음식이 맛있어요\n가이드...
118,5,min******,가족여행으로 좋아요,30대,2023.12.27,알차고 즐거운여행였습니다.,호텔에서 쉬기 좋아요\n즐길 거리가 다양해요\n현지 음식이 맛있어요\n가격만큼 가치...


In [17]:
20 % 10

0